# Tech Intro

## Main tables

In [635]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [636]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = deal_list[2] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [637]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1120 01:06:40.721570 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1120 01:06:40.722877 22264 sched.cpp:241] Version: 1.11.0
I1120 01:06:40.723147 10569 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1120 01:06:40.723241 10569 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1120 01:06:40.724750 10573 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75610


## Read the spec-file from Hadoop and show some data sample

In [638]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+-----+-------+----------+----------+----------+----------+---------+--------+----------+----------+---------------+-----------+--------------+---------------------------+-----------------+----------------+-----------+-----------------------+------------+------------+----------+--------------------+------+------------+---------+----------------+------------------+-----------------+-------------------------+---------------+-------------------------+-----------+--------+----------+--------------+---------+-------+-----------------+----------------------+------------+------------------+--------------+--------------------+------------+------------+--------------+---------+----------------+-----------+------+---------------+------------------------------+---------+----------------------------------------+
|id   |deal_id|created_by|created_at|updated_by|updated_at|deal_type|deal_vid|deal_date |pdkp_date |object_by_order|object_from|join_deal_with|seller_recomend            |customer_recomend

## Show columns' properties

In [639]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- deal_id: string (nullable = true)
 |-- created_by: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- updated_by: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- deal_type: string (nullable = true)
 |-- deal_vid: string (nullable = true)
 |-- deal_date: string (nullable = true)
 |-- pdkp_date: string (nullable = true)
 |-- object_by_order: string (nullable = true)
 |-- object_from: string (nullable = true)
 |-- join_deal_with: string (nullable = true)
 |-- seller_recomend: string (nullable = true)
 |-- customer_recomend: string (nullable = true)
 |-- seller_recomends: string (nullable = true)
 |-- object_type: string (nullable = true)
 |-- address: string (nullable = true)
 |-- cash_payment: string (nullable = true)
 |-- credit_bank: string (nullable = true)
 |-- prepayment: string (nullable = true)
 |-- first_payment_option: string (nullable = true)
 |-- credit: string (nullable = tr

## Total rows (count)

In [640]:
eda_df.count()

35346

## Simple func to execute and show the result of the SQL-instruction

In [641]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [642]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [643]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|35346   |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [644]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,deal_id,created_by,created_at,updated_by,updated_at,deal_type,deal_vid,deal_date,pdkp_date,...,seller_agent,customer_agent,recomends,recomend_payment,other_agent,status,recomend_seller,mebel,insurance,comment
0,0,0,0,0,0,0,18484,22442,23792,20897,...,25009,27036,30500,22162,0,0,0,0,0,0


In [645]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,deal_id,created_by,created_at,updated_by,updated_at,deal_type,deal_vid,deal_date,pdkp_date,...,seller_agent,customer_agent,recomends,recomend_payment,other_agent,status,recomend_seller,mebel,insurance,comment
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,32369,0,0,3,0


In [646]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,deal_id,created_by,created_at,updated_by,updated_at,deal_type,deal_vid,deal_date,pdkp_date,...,seller_agent,customer_agent,recomends,recomend_payment,other_agent,status,recomend_seller,mebel,insurance,comment
0,0,0,0,0,0,0,0,0,0,0,...,416,550,537,308,31833,0,29188,24639,32029,29562


In [647]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
deal_id,0,0,0
created_by,0,0,0
created_at,0,0,0
updated_by,0,0,0
updated_at,0,0,0
deal_type,18484,0,0
deal_vid,22442,0,0
deal_date,23792,0,0
pdkp_date,20897,0,0


In [648]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [649]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
deal_id,0.000,0.000,0.000
created_by,0.000,0.000,0.000
created_at,0.000,0.000,0.000
updated_by,0.000,0.000,0.000
updated_at,0.000,0.000,0.000
deal_type,52.294,0.000,0.000
deal_vid,63.492,0.000,0.000
deal_date,67.312,0.000,0.000
pdkp_date,59.121,0.000,0.000


### SQL-instructions

In [650]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|35346             |
+------------------+

+-----------------------+
|count(DISTINCT deal_id)|
+-----------------------+
|35300                  |
+-----------------------+

+--------------------------+
|count(DISTINCT created_by)|
+--------------------------+
|2210                      |
+--------------------------+



+--------------------------+
|count(DISTINCT created_at)|
+--------------------------+
|35302                     |
+--------------------------+

+--------------------------+
|count(DISTINCT updated_by)|
+--------------------------+
|2015                      |
+--------------------------+

+--------------------------+
|count(DISTINCT updated_at)|
+--------------------------+
|35304                     |
+--------------------------+

+-------------------------+
|count(DISTINCT deal_type)|
+-------------------------+
|4                        |
+-------------------------+

+------------------------+
|count(DISTINCT deal_vid)|
+------------------------+
|3                       |
+------------------------+

+-------------------------+
|count(DISTINCT deal_date)|
+-------------------------+
|1571                     |
+-------------------------+

+-------------------------+
|count(DISTINCT pdkp_date)|
+-------------------------+
|1616                     |
+-------------------------+

+--

In [651]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [652]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute_template(col)

+----+
|id  |
+----+
|9999|
|9998|
|9997|
|9996|
|9995|
|9994|
|9993|
|9992|
|9991|
|9990|
|999 |
|9989|
|9988|
|9987|
|9986|
|9985|
|9984|
|9983|
|9982|
|9981|
+----+
only showing top 20 rows

None


+-----+
|id   |
+-----+
|1    |
|10   |
|100  |
|1000 |
|10000|
|10001|
|10002|
|10003|
|10004|
|10005|
|10006|
|10007|
|10008|
|10009|
|1001 |
|10010|
|10011|
|10012|
|10013|
|10014|
+-----+
only showing top 20 rows

+-------+
|deal_id|
+-------+
|99980  |
|99976  |
|99966  |
|99919  |
|99882  |
|99860  |
|99855  |
|99829  |
|99826  |
|99795  |
|99770  |
|99729  |
|99723  |
|99709  |
|99708  |
|99673  |
|99669  |
|99665  |
|99661  |
|99652  |
+-------+
only showing top 20 rows

None
+-------+
|deal_id|
+-------+
|100001 |
|100020 |
|100027 |
|100039 |
|100053 |
|100060 |
|100121 |
|100137 |
|100158 |
|100167 |
|100172 |
|100212 |
|100275 |
|100308 |
|100315 |
|100321 |
|100323 |
|100352 |
|10037  |
|100371 |
+-------+
only showing top 20 rows

+----------+
|created_by|
+----------+
|996       |
|98        |
|9679      |
|9677      |
|9671      |
|9613      |
|9607      |
|9588      |
|9582      |
|95        |
|942       |
|928       |
|92        |
|91        |
|905       |
|903       

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [634]:
spark.stop()

I1120 01:04:44.469843 10390 sched.cpp:2174] Asked to stop the driver
I1120 01:04:44.469959 10546 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75606
